# Data Analysis made easy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import html5lib
%matplotlib inline

### Reading data using read_html

#### read_html will be handy when you want to extract the table data from html page. read_html will result a list containing one dataframe for each table in the html page

In [ ]:
urls_dict = {"http://www.siamindia.com/statistics.aspx?mpgid=8&pgidtrail=14":"Society of Indian Automobile Manufactures.html",
            "http://www.valueresearchonline.com/funds/newsnapshot.asp?schemecode=3019":"Value Research Online.html",
            "http://www.fdic.gov/bank/individual/failed/banklist.html":"FDIC_ Failed Bank List.html"}

In [ ]:
def get_url(url):
    try:
        import requests
        r = requests.get(url,timeout=2)
        return url
    except:
        return urls_dict[url]

In [ ]:
auto_sales_url = get_url("http://www.siamindia.com/statistics.aspx?mpgid=8&pgidtrail=14")
table_list = pd.read_html(auto_sales_url,header=0)
auto_sales_df = table_list[0]

In [ ]:
print(auto_sales_url)

In [ ]:
len(table_list)

In [ ]:
auto_sales_df.head()

In [ ]:
auto_sales_df = auto_sales_df.set_index("Category")

In [ ]:
auto_sales_df.head()

In [ ]:
# finding the shape of the data frame ( rows,  columns)
auto_sales_df.shape

In [ ]:
# extractig the column data
auto_sales_df["2010-11"]

In [ ]:
auto_sales_df.head()

In [ ]:
auto_sales_df.index

In [ ]:
auto_sales_df.columns

In [ ]:
# extracting the row data
auto_sales_df.loc["Passenger Vehicles"]

In [ ]:
auto_sales_df.dtypes

In [ ]:
auto_sales_df = auto_sales_df.drop("Grand Total", axis=0)
auto_sales_df.head()

In [ ]:
auto_sales_graph = auto_sales_df.T.plot(kind='barh',figsize=(14,6),title="Auto Sales")

In [ ]:
# html page containing multiple tables in the same html
funds_url = get_url('http://www.valueresearchonline.com/funds/newsnapshot.asp?schemecode=3019')

In [ ]:
print(funds_url)

In [ ]:
df_list = pd.read_html(funds_url)

In [ ]:
len(df_list)

In [ ]:
df_list[0].head()

In [ ]:
# Extracting one of the multiple tables by matching to a string
#filtered_df_list = pd.read_html(funds_url,match='Rank within Category')
filtered_df_list = pd.read_html(funds_url,match='Rank within Category',header=0,index_col=0)
#skiprows

In [ ]:
len(filtered_df_list)

In [ ]:
filtered_df_list[0].head()

In [ ]:
# Copying the data frame explictly into another dataframe
# selecting only the required columns
df = filtered_df_list[0].iloc[:,[0,1,2,4,5]].copy()
df.head()

In [ ]:
df.dtypes

In [ ]:
# Converting column data types
for column in df.columns:    
    df[column] = df[column].astype(np.float16)

In [ ]:
df.dtypes

In [ ]:
df.loc[["Fund","Category","Nifty Free Float Midcap 100"]].plot(figsize=(12,6),kind='bar',rot=0)
#df.loc[["Fund","Category","Nifty Midcap 100"]].T.plot(figsize=(12,6),kind='bar',rot=0)

In [ ]:
# using attrs in read_html for filtering the data
df_attr_list = pd.read_html(funds_url,
                            attrs={"id":"fund-snapshot-peer-comparison-table"},
                            header=0,index_col=0)

In [ ]:
df_attr_list[0].head()

In [ ]:
df = df_attr_list[0].dropna(axis=1).copy()
df.head()

In [ ]:
df["Assets (Cr)"] = df["Assets (Cr)"].astype(np.int32)

In [ ]:
df[["1-Year Ret","3-Year Ret","5-Year Ret"]].T.plot(figsize=(18,6),kind='bar',rot=0)

In [ ]:
bank_url = get_url('http://www.fdic.gov/bank/individual/failed/banklist.html')
#bank_df_list = pd.read_html(url)
bank_df_list = pd.read_html(bank_url,parse_dates=[5,6,9])
bank_df = bank_df_list[0]

In [ ]:
print(bank_url)

In [ ]:
bank_df.head(2)

In [ ]:
# let us see if there are records with same Bank Name
bank_df_name_duplicates = bank_df[bank_df["Bank Name"].duplicated()]

In [ ]:
# Check the data types for date columns
bank_df.dtypes

In [ ]:
# filtering for duplicated record and sorting by Bank Name
bank_df[bank_df["Bank Name"].isin(bank_df_name_duplicates["Bank Name"])].sort_values(by="Bank Name").head()

In [ ]:
# It looks like the Bank Name and City makes the unique combination
# Let's check if there are any duplicates with this combination
bank_df[bank_df[["Bank Name","City"]].duplicated()]

In [ ]:
print bank_df["Closing Date"].min()
print bank_df["Closing Date"].max()

In [ ]:
bank_df["Closing Date"].map(lambda x: x.year).head()

In [ ]:
# Let us find the number of banks closed in each year using map
bank_df["Closing Date"].map(lambda x: x.year).value_counts().plot(kind='bar')

In [ ]:
# Let us find the number of banks closed in each year using groupby
bank_df_group = bank_df.groupby([bank_df["Closing Date"].map(lambda x: x.year)])
type(bank_df_group)

In [ ]:
# plot the number of banks closed by each year
bank_df_group.count()["Bank Name"].plot(kind='bar')

In [ ]:
# Number of banks closed by each state between 2000 and 2015
bank_df["ST"].value_counts().plot(kind='bar',figsize=(20,4))

In [ ]:
# get the state names from another url
states_df = pd.read_html("http://www.infoplease.com/ipa/A0110468.html", attrs={"id":"A0110469"},header=0)[0]

In [ ]:
states_df.columns

In [ ]:
# check the dataframe for GA state
states_df[states_df["Postal Code"]=="GA"]

In [ ]:
# Join the state names to bank_df dataframe using merge
bank_df_state_names= pd.merge(bank_df,states_df,
                              how="left",
                              left_on="ST",
                              right_on="Postal Code",
                              sort=False)

In [ ]:
bank_df_state_names.head(2)

In [ ]:
# number of banks closed by state name
bank_df_state_names["State"].value_counts().plot(kind='bar',figsize=(20,4))

### Reading data using read_clipboard

#### somtimes if you want to bring data copied in clip board into pandas dataframe, you can use read_clipboard

In [ ]:
#parse_dates=[3]
#top10_funds_df = pd.read_clipboard()
top10_funds_df = pd.read_clipboard(parse_dates=[3])
top10_funds_df.head()
#You may need to install xclip or xsel (with gtk or PyQt4 modules) on Linux to use these methods.

In [ ]:
top10_funds_df.dtypes

In [ ]:
column_names = ['Table Name','State Code','District Code','Area Name','Age',
                    'Persons-Total','Males-Total','Females-Total',
                    'Persons-Rural','Males-Rural','Females-Rural',
                    'Persons-Urban','Males-Urban','Females-Urban']
excel_df = pd.read_excel("DDW-0000C-13.xls",skiprows=6, names = column_names)
#excel_df.columns = column_names

In [ ]:
excel_df.head()

In [ ]:
# Filtering data for karnata state
req_columns = ["Age","Males-Rural","Females-Rural","Males-Urban","Females-Urban"]
kar_filter = (excel_df["Area Name"] == "State - KARNATAKA (29)") & (excel_df["Age"] != "All ages") & (excel_df["Age"] != "Age not stated")
excel_kar_df = excel_df[kar_filter][req_columns].copy()

In [ ]:
excel_kar_df.head(2)

In [ ]:
excel_kar_df.dtypes

In [ ]:
# find unique values in Age Column
excel_kar_df.Age.unique()

In [ ]:
# Converting Age to integer data type
excel_kar_df.Age = excel_kar_df.Age.apply(lambda x:int(str(x).replace('+','')))

In [ ]:
excel_kar_df.dtypes

In [ ]:
# let's plot the graph and observe
# save the graph to a file
kar_graph = excel_kar_df.set_index("Age").plot(figsize=(20,10))
#kar_graph  = excel_kar_df.set_index("Age").plot(figsize=(16,10),subplots=True)
#kar_graph = excel_kar_df.set_index("Age").plot(figsize=(16,10),subplots=True,ylim=(0,600000))
plt.savefig("kar_graph.jpeg")

In [ ]:
# let us observe the Age not stated by state
age_filter = (excel_df["Age"] == "Age not stated") & (excel_df["Area Name"] != "India")
required_columns = ["Area Name","Males-Rural","Females-Rural","Males-Urban","Females-Urban"]
excel_df_age_ns = excel_df[age_filter][required_columns]
excel_df_age_ns = excel_df_age_ns.set_index("Area Name")
age_graph = excel_df_age_ns.plot(kind='bar',figsize=(18,6))

In [ ]:
# let us look at the distribution
kar_age_graph = excel_kar_df.set_index("Age").plot(figsize=(12,6),kind='box')

In [ ]:
# describe gives a stats summary of all of the numeric fields
excel_kar_df.describe()

### Reading data using read_csv

#### One of the very commonly used method is read_csv

In [ ]:
# district wise rail fall data from 1951 to 2000
data = pd.read_csv("distirct_railfaill_1951_2000.csv")

In [ ]:
data.head(2)

In [ ]:
# copy the Karnataka rain fall data into a separate dataframe
data_kar = data[data["STATE/UT"]=="KARNATAKA"].copy()

In [ ]:
# drop STATE_UT column and set DISTRICT as the index
data_kar = data_kar.drop("STATE/UT", axis=1)
data_kar = data_kar.set_index("DISTRICT")

In [ ]:
# Keep only required columns which are monts from JAN to DEC
data_kar = data_kar[[u'JAN', u'FEB', u'MAR', u'APR', u'MAY', u'JUN', u'JUL', u'AUG', u'SEP',
       u'OCT', u'NOV', u'DEC']]

In [ ]:
data_kar.head()

In [ ]:
data_kar.describe()

In [ ]:
# Let us create the axes plot obect and use set_ylable and set_title methods
#data_kar.plot?
ax = data_kar.describe().T.plot(colormap=plt.cm.Accent_r ,figsize=(12,6),grid=True)
ax.set_ylabel("rain fall(mm)",fontdict={'size':12})
ax.set_title("Rail fall in Karnata(1951-2000)",fontdict={'size':12})

### Using default values while reading csv file

In [ ]:
# let us look at a bit complicated csv file and see how we can use all of the options
# available with read_csv method
!head -10 GLB.Ts+dSST.txt

In [ ]:
#pd.read_csv?
required_columns = ["Year","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
glb_temp_df = pd.read_csv("GLB.Ts+dSST.txt",skiprows=7,usecols=required_columns,
                      skip_footer=12,engine='python',sep="\s+",na_values=['****','***'],
                      index_col=0
                      )

In [ ]:
glb_temp_df.head()

In [ ]:
# there are headers in between the data sets. Let us remove these headers from the file
# please note that index can have duplicates
glb_temp_df.loc["Year"]

In [ ]:
glb_temp_df = glb_temp_df[glb_temp_df["Jan"] != "Jan"]
glb_temp_df.shape

In [ ]:
glb_temp_df.index

In [ ]:
glb_temp_df.tail()

In [ ]:
glb_temp_df.dtypes

In [ ]:
# let us convert the teamperatue columns to int
# let fill the NA values with forward fill
for col in glb_temp_df.columns:
    glb_temp_df[col] = glb_temp_df[col].ffill().astype(np.int64)

In [ ]:
glb_temp_df.dtypes

In [ ]:
glb_temp_df.tail()

In [ ]:
# mean temp by year
glb_temp_df.mean(axis=1).head()

In [ ]:
# mean temp by month
glb_temp_df.mean(axis=0).head()

In [ ]:
glb_temp_df.mean(axis=0).plot()

In [ ]:
glb_temp_df.mean(axis=1).plot(figsize=(20,4))

### Exploring groupby functionality

In [ ]:
# Let us take the rain fall data
data.head(2)

In [ ]:
# first create the group by object
data_gb = data.groupby("STATE/UT")

In [ ]:
type(data_gb)

In [ ]:
len(data_gb)

In [ ]:
len(data["STATE/UT"].unique())

In [ ]:
for name,group in data_gb:
    print(name)

In [ ]:
#gettig one groups data
type(data_gb.get_group("KARNATAKA"))

In [ ]:
data_gb_mean = data_gb.mean()

In [ ]:
type(data_gb_mean)

In [ ]:
data_gb_mean.head(2)

In [ ]:
#data_gb.groups['ANDAMAN And NICOBAR ISLANDS']
data_gb.get_group('ANDAMAN And NICOBAR ISLANDS')

In [ ]:
months_q1 = ["JAN","FEB","MAR"]
months_q2 = ["APR","MAY","JUN"]
months_q3 = ["JUL","AUG","SEP"]
months_q4 = ["OCT","NOV","DEC"]
#months = ["MAR","JUN","SEP","DEC"]
ax= data_gb[months_q4].sum().plot(figsize=(20,8),kind='bar',subplots=True,legend=False,ylim=(0,6500))

In [ ]:
data_gb_quarter = data_gb.sum()

In [ ]:
quarters = {'Q1':months_q1,'Q2':months_q2,'Q3':months_q3,'Q4':months_q4}
for quarter in quarters:
    #print quarters[quarter]
    data_gb_quarter[quarter] = (data_gb_quarter[quarters[quarter]]).mean(axis=1)

In [ ]:
data_gb_quarter[['Q1','Q2','Q3','Q4']].head()

In [ ]:
ax = data_gb_quarter[['Q1','Q2','Q3','Q4']].plot(kind='bar',figsize=(16,6))